In [1]:
import math
import neat
import numpy as np
import pygame
import random
import sys
#import cv2

pygame 2.6.0 (SDL 2.28.4, Python 3.12.2)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [ ]:


'''def find_start_line(track_image_path, start_line_image_path):
    track_image = cv2.imread(track_image_path)
    start_line_image = cv2.imread(start_line_image_path)
    gray_track = cv2.cvtColor(track_image, cv2.COLOR_BGR2GRAY)
    gray_start_line = cv2.cvtColor(start_line_image, cv2.COLOR_BGR2GRAY)
    result = cv2.matchTemplate(gray_track, gray_start_line, cv2.TM_CCOEFF_NORMED)
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)
    start_line_x, start_line_y = max_loc
    print(f"Start line coordinates: ({start_line_x}, {start_line_y})")

# Example usage
track_image_path = 'assets/ita.png'
start_line_image_path = 'assets/start_line2.png'
find_start_line(track_image_path, start_line_image_path)'''

In [36]:

pygame.init()

WIDTH = 1366
HEIGHT = 768

# Temporary dictionary for start line coordinates
s_coord = {
    "bah.png": (806, 664),
    "bel.png": (491, 325),
    "ita.png": (908, 552),
    "gbr.png": (902, 451),
    "jpn.png": (599, 614),
    "rand.png": (776, 604),
    "rand_hard.png": (876, 647)
}

class Car:
    def __init__(self):
        self.sprite = pygame.image.load("assets/w10_min.png").convert_alpha()
        self.rotated_sprite = self.sprite
        self.track_name = "bah.png"
        self.track = pygame.image.load("assets/" + self.track_name).convert()
        
        # Car dimensions
        car_width = 52
        car_height = 29
        
        # Start position from the dictionary
        start_position = s_coord[self.track_name]
        
        # Adjusted position to place the center of the car at the start position
        self.s_position = (start_position[0], start_position[1])
        
        self.speed_set = False
        self.speed = 0
        self.angle = 0
        self.center = [start_position[0]+26, start_position[1]+14.5]
        self.sensors = []
        self.isAlive = True
        self.dist = 0
        self.time = 0

    def draw(self, screen):
        screen.blit(self.track, (0, 0))  # Draw the track first
        screen.blit(self.rotated_sprite, self.s_position)
        self.show_sensors(screen)
    
    def show_sensors(self, screen):
        for sensor in self.sensors:
            pos = sensor[0]
            pygame.draw.line(screen, (0, 255, 0), self.center, pos, 1)
            pygame.draw.circle(screen, (255, 0, 0), pos, 4)

    def check_collision(self):
        self.isAlive = True
        for p in self.corners:
            x, y = int(p[0]), int(p[1])
            if 0 <= x < WIDTH and 0 <= y < HEIGHT:
                if self.track.get_at((x, y)) == (255, 255, 255, 255):
                    self.isAlive = False
                    break

    def check_sensor(self, degree): 
        length = 0
        x, y = 0, 0

        while length < 150:
            length += 1
            x = int(self.center[0] + math.cos(math.radians(360 - (self.angle + degree))) * length)
            y = int(self.center[1] + math.sin(math.radians(360 - (self.angle + degree))) * length)
            if 0 <= x < WIDTH and 0 <= y < HEIGHT:
                if self.track.get_at((x, y)) == (255, 255, 255, 255):
                    break

        self.sensors.append(([x, y], length))

    def update(self):
        if not self.speed_set:
            self.speed = 0.5
            self.speed_set = True

        self.rotated_sprite = self.rotate_center(self.sprite, self.angle)
        
        self.s_position = list(self.s_position) 
        self.s_position[0] -= math.cos(math.radians(360 - self.angle)) * self.speed
        self.s_position[0] = max(self.s_position[0], 4)
        self.s_position[0] = min(self.s_position[0], WIDTH - 39)

        self.s_position[1] -= math.sin(math.radians(360-self.angle)) * self.speed
        self.s_position[1] = max(self.s_position[1], 4)
        self.s_position[1] = min(self.s_position[1], HEIGHT - 62)

        self.dist += self.speed
        self.time += 1

        self.center = (self.s_position[0] + 26, self.s_position[1] + 14.5)

        length = 0.5 * 52

        left_top = [self.center[0] + math.cos(math.radians(360 - (self.angle + 30))) * length, self.center[1] + math.sin(math.radians(360 - (self.angle + 30))) * length]
        right_top = [self.center[0] + math.cos(math.radians(360 - (self.angle + 150))) * length, self.center[1] + math.sin(math.radians(360 - (self.angle + 150))) * length]
        left_bottom = [self.center[0] + math.cos(math.radians(360 - (self.angle + 210))) * length, self.center[1] + math.sin(math.radians(360 - (self.angle + 210))) * length]
        right_bottom = [self.center[0] + math.cos(math.radians(360 - (self.angle + 330))) * length, self.center[1] + math.sin(math.radians(360 - (self.angle + 330))) * length]
        self.corners = [left_top, right_top, left_bottom, right_bottom]

        self.check_collision()
        self.sensors.clear()

        for d in range(-270, -45, 45):
            self.check_sensor(d)

    def rotate_center(self, image, angle):
        rect = image.get_rect()
        rotated_image = pygame.transform.rotate(image, angle)
        rotated_rect = rect.copy()
        rotated_rect.center = rotated_image.get_rect().center
        rotated_image = rotated_image.subsurface(rotated_rect).copy()
        return rotated_image

    def fetch(self):
        sensors = self.sensors
        ret = [0, 0, 0, 0, 0]
        for i, s in enumerate(sensors):
            ret[i] = int(s[1]/30)
        return ret

    def is_alive(self):
        return self.isAlive
    
    def reward(self):
        return self.dist/26
    

    

# Initialize Pygame screen
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Car Simulation Test")

# Create a Car instance
car = Car()

# Main loop
running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    # Clear the screen
    screen.fill((0, 0, 0))

    # Update the car
    car.update()

    # Draw the car
    car.draw(screen)

    # Update the display
    pygame.display.flip()

pygame.quit()

In [18]:
'''
# Main function
def main():
    screen = pygame.display.set_mode((WIDTH, HEIGHT))
    pygame.display.set_caption("Car on Track")

    track_name = "bel.png"  
    car = Car(track_name)

    screen.blit(car.track, (0, 0))
    car.draw(screen)

    pygame.display.flip()

    running = True
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False


    pygame.time.wait(5000)

    pygame.quit()

if __name__ == "__main__":
    main()'''


In [ ]:
def simulate(genomes, config):
    nets = []
    cars = []
    for _, g in genomes:
        net = neat.nn.FeedForwardNetwork.create(g, config)
        nets.append(net)
        cars.append(Car())
        g.fitness = 0

    screen = pygame.display.set_mode((WIDTH, HEIGHT))
    pygame.display.set_caption("Car on Track")

    track_name = "bel.png"
    car = Car()

    screen.blit(car.track, (0, 0))
    car.draw(screen)

    pygame.display.flip()

    running = True
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

        for x, car in enumerate(cars):
            output = nets[x].activate(car.fetch())
            car.angle += output.index(max(output)) * 45
            car.update()
            car.draw(screen)

            if not car.is_alive():
                ge[x].fitness -= 1
                cars.pop(x)
                nets.pop(x)
                ge.pop(x)

        pygame.display.flip()

    pygame.quit()